In [ ]:
import os
import tqdm
import pandas as pd
import numpy as np
from pathlib import Path

import bw2data as bd
import bw2io as bi
import bw2calc as bc
import bw2analyzer as bwa
import time

In [ ]:
bd.projects

In [ ]:
bd.projects.set_current(name='corbu')

In [ ]:
bd.databases

In [ ]:
del bd.databases['corbu_custom_processes']

In [ ]:
my_db = bi.ExcelImporter(Path.cwd().parent.joinpath("./data/lca_databases/lci_ciment_beton.xlsx"))

In [ ]:
my_db.apply_strategies()

In [ ]:
my_db.match_database(
    "ecoinvent-3.9.1-cutoff", fields=["name", "unit", "reference product", "location"]
)

In [ ]:
my_db.match_database(
    "ecoinvent-3.9.1-biosphere", fields=["name", "unit", "location"]
)

In [ ]:
my_db.statistics()

In [ ]:
my_db

In [ ]:
my_db.write_database()

In [ ]:
parois_db = bd.Database('parois')

In [ ]:
print([act for act in parois_db])

In [ ]:
housing_db = bd.Database('housing_structures')
eidb = bd.Database('ecoinvent-3.9.1-cutoff')

In [ ]:
# Collect relevant processes
# timber
process_glt = [
    act for act in bd.Database("bois") \
        if 'glued laminated timber, A-C'== act['name']
][0]
process_clt = [
    act for act in bd.Database("bois") \
        if 'cross laminated timber, A-C' == act['name']
][0]
process_wood_joist = [
    act for act in bd.Database("bois") \
        if 'wood joist, A-C' == act['name']
][0]
# Steel
process_reinforcing_steel = [
    act for act in bd.Database("acier") \
        if 'reinforcing steel, A-C' == act['name']
][0]
process_sheet_steel = [
    act for act in bd.Database("acier") \
        if 'sheet steel, A-C' == act['name']
][0]
process_standard_steel = [
    act for act in bd.Database("acier") \
        if 'standard steel, A-C' == act['name']
][0]
process_plaster = [
    act for act in bd.Database("acier") if 'plaster' == act['name']
][0]
process_stone_wool = [
    act for act in bd.Database("acier") if 'stone wool' == act['name']
][0]
# Concrete
process_concrete_standard = [
    act for act in bd.Database("ciment_beton") \
        if 'concrete, cement CEM II/A, building construction, A-C,' \
            + ' economic allocation' == act['name']
][0]
process_concrete_screed = [
    act for act in bd.Database("ciment_beton") \
        if 'concrete, cement CEM II/A, for lean concrete, A-C,' \
            + ' economic allocation' == act['name']
][0]
process_concrete_fondation = [
    act for act in bd.Database("ciment_beton") if \
        'concrete, cement CEM II/A, civil engineering, A-C,' \
            + ' economic allocation' == act['name']][0]

process_hvac = [
    a for a in eidb.search('heat production, air-water heat pump') \
        if 'Europe' in a['location']
][0]

process_poly = [
    a for a in parois_db.search('Polystyrène')
][0]

process_vitrage = [
    a for a in parois_db.search('Triple vitrage')
][0]

process_plaster = [
    a for a in parois_db.search('Plâtre')
][0]

# Store material processes functional unit conversion coeffs
# (to translate kg to m3 when needed)
# Get weights of for 1 m3 of each material
p_glt = 490
p_clt = 480
p_bois_massif = 480
p_cemii = 2400
p_osb = 616
p_acier = 7850
processes = {
    'standard_steel': {
        'names': ['Acier-S235'],
        'process': process_standard_steel,
        'FU_conversion_coeff': 1
    },
    'glt': {
        'names': ['Bois-GL24h'],
        'process': process_glt,
        'FU_conversion_coeff': 1/p_glt
    },
    'standard_concrete': {
        'names': ['Béton CEMII', 'Dalle Béton'],
        'process': process_concrete_standard,
        'FU_conversion_coeff': 1/p_cemii
    },
    'reinforcing_steel': {
        'names': ['Ferraillage'],
        'process': process_reinforcing_steel,
        'FU_conversion_coeff': 1
    },
    'sheet_steel': {
        'names': ['Acier-Assemblage', 'Tôle'],
        'process': process_sheet_steel,
        'FU_conversion_coeff': 1
    },
    'concrete_screed': {
        'names': ['Chape Béton'],
        'process': process_concrete_screed,
        'FU_conversion_coeff': 1/p_cemii
    },
    'clt': {
        'names': ['-CLT'],
        'process': process_clt,
        'FU_conversion_coeff': 1/p_clt
    },
    'wood_joist': {
        'names': ['Bois-C18', 'Solives bois'],
        'process': process_wood_joist,
        'FU_conversion_coeff': 1/p_bois_massif
    },
    'plaster': {
        'names': ['Platre'],
        'process': process_plaster,
        'FU_conversion_coeff': 1
    },
    'concrete_foundation': {
        'names': ['Fondation-Beton'],
        'process': process_concrete_fondation,
        'FU_conversion_coeff': 1/p_cemii
    },
    'stone_wool': {
        'names': ['Flocage'],
        'process': process_stone_wool,
        'FU_conversion_coeff': 1
    },
    'hvac': {
        'names': ['Total HVAC consumption (kWh/year)'],
        'process': process_hvac,
        'FU_conversion_coeff': 3.6
    },
    'polystyrene': {
        'names': ['Polystyrène'],
        'process': process_poly,
        'FU_conversion_coeff': 1
    },
    'triple_glazing': {
        'names': ['Triple vitrage'],
        'process': process_vitrage,
        'FU_conversion_coeff': 1
    },
    'plaster': {
        'names': ['Plâtre'],
        'process': process_plaster,
        'FU_conversion_coeff': 1
    }
}



In [ ]:
process_plaster

In [ ]:
recipe_methods = [
    m for m in bd.methods if 'recipe 2016 v1.03' in m[1].lower()
    and '(h)' in m[1].lower() and 'no lt' not in m[1].lower() and "ecoinvent-3.9.1" in m[0].lower()
]
recipe_methods

In [ ]:
len(recipe_methods)

In [ ]:
# Select methods
recipe_methods = [
    m for m in bd.methods if 'recipe 2016 v1.03' in m[1].lower()
    and '(h)' in m[1].lower() and 'no lt' not in m[1].lower() and "ecoinvent-3.9.1" in m[0].lower()
]

# Perform LCA
functional_units = {}
for process_dict in processes.values():
    functional_units[process_dict['process']['name']] = {
        process_dict['process'].id: 1
    }

config = {
    'impact_categories': recipe_methods
}

data_objs = bd.get_multilca_data_objs(
    functional_units=functional_units,
    method_config=config
)
mlca = bc.MultiLCA(
    demands=functional_units,
    method_config=config,
    data_objs=data_objs,
    use_distributions=True
)
mlca.lci()
mlca.lcia()
scores = [mlca.scores for _ in tqdm.tqdm(zip(mlca, range(10000)), total=10000)]

In [ ]:
scores

In [ ]:
# Instantiate results dict
results_dict = {}
# Iterate over material
for mat in functional_units.keys():
    # Iterate over methods
    for m in recipe_methods:
        results_dict[f'{mat}_{m}'] = []

# Iterate over scores and save results
for res in scores:
    for key, val in res.items():
        results_dict[f'{key[-1]}_{key[0]}'].append(val)

# Compile results in a df and save them to csv
df = pd.DataFrame(results_dict)
df.index.name = 'Run ID'
df

In [ ]:
df.to_csv('prerun_lca_MC10000.csv')